In [ ]:
import azureml.core
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)


In [ ]:
!pip install --upgrade azureml-sdk

## Initialize workspace

In [ ]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()

print(ws.name, ws.location, ws.resource_group, ws.location, sep='\t')


## Create an experiment

In [ ]:
experiment_name = 'diabetes_exp'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

exp


## Upload dataset

In [ ]:
from azureml.core import Dataset

dataset = Dataset.get_by_name(ws, name='diabetes')


In [ ]:
diabetes = dataset.to_pandas_dataframe().drop("Path", axis=1)

In [ ]:
diabetes.head()

In [ ]:
diabetes.info()

In [ ]:
diabetes.describe()

In [ ]:
features_names = ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']
target = "Y"

## Train the model

In [ ]:
# Separate train and test data

from sklearn.model_selection import train_test_split

X = diabetes.drop(target, axis=1)
y = diabetes["Y"].values.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# There are three steps to model something with sklearn

from sklearn.linear_model import LinearRegression

# 1. Set up the model
model = LinearRegression()
# 2. Use fit
model.fit(X_train, y_train)
# 3. Check the score
model.score(X_test, y_test)


In [ ]:
import matplotlib.pylab as plt
import numpy as np
%matplotlib inline

# plot prediction and actual data
y_pred = model.predict(X_test) 
plt.plot(y_test, y_pred, '.')

# plot a line, a perfit predict would all fall on this line
x = np.linspace(0, 330, 100)
y = x
plt.plot(x, y)
plt.show()


In [ ]:
from sklearn.externals import joblib
import pickle

os.makedirs('models', exist_ok=True)

# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='models/diabetes_regression_model.pkl')


## Register the model

In [ ]:
# sckit learn version
from sklearn import __version__
print(sklearn.__version__)

In [ ]:
# Register the model (from the locally train)

from azureml.core.model import Model

model = Model.register(model_path="models/diabetes_regression_model.pkl",
                       model_name="diabetes_regression_model",
                       model_framework=Model.Framework.SCIKITLEARN,  # Framework used to create the model.
                       model_framework_version='0.20.3',             # Version of scikit-learn used to create the model.
                       tags={'area': "diabetes", 'type': "regression"},
                       description="Linear regression model to predict diabetes",
                       workspace=ws)


In [ ]:
# load an existing model

from azureml.core.model import Model

model = Model(ws, "diabetes_regression_model")

print(model.id, model.name, model.version, sep='\t')


### Define model with input and output datasets

In [ ]:
from azureml.core import Dataset

input_dataset = Dataset.get_by_name(ws, name='diabetes').drop_columns("y")
output_dataset = Dataset.get_by_name(ws, name='diabetes').keep_columns("y", validate=False)

In [ ]:
# Register model with sample inpout and output

from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

model = Model.register(workspace=ws,
                       model_name="diabetes_reg_model_wsample",                # Name of the registered model in your workspace.
                       model_path="models/diabetes_regression_model.pkl",      # Local file to upload and register as a model.
                       model_framework=Model.Framework.SCIKITLEARN,            # Framework used to create the model.
                       model_framework_version='0.20.3',                       # Version of scikit-learn used to create the model.
                       sample_input_dataset=input_dataset,
                       sample_output_dataset=output_dataset,
                       resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5),
                       description='Linear regression model to predict diabetes progression.',
                       tags={'area': 'diabetes', 'type': 'regression'})

print('Name:', model.name)
print('Version:', model.version)

In [ ]:
# load an existing model

from azureml.core.model import Model

model = Model(ws, "diabetes_reg_model_wsample")
model

#VOIR LA FONCTION get_model_path

## Deploy in a default environment

In [ ]:
# Use a default environment
# nécessite d'avoir précisé l'environnement lors de l'enregistrement du modèle
# Default environments are only provided for these frameworks: ['Onnx', 'ScikitLearn', 'TensorFlow']."

from azureml.core import Webservice
from azureml.exceptions import WebserviceException

service_name = 'diabetes-service'

# Remove any existing service under the same name.
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass

service = Model.deploy(ws, service_name, [model])
service.wait_for_deployment(show_output=True)

In [ ]:
# reload the model

from azureml.core import Webservice

service_name = 'diabetes-service'
service = Webservice(ws, service_name)

print(service.name, service.scoring_uri, sep='\t')

## Test the service

In [ ]:
import json

input_payload = json.dumps({
    'data': [
        [59, 2, 32.1, 101.0, 157, 93.2, 38.0, 4.0, 4.8598, 87],
        [69, 2, 32.1, 101.0, 157, 93.2, 38.0, 4.0, 4.8598, 87]
    ],
    'method': 'predict'  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(input_payload)

print(output)

In [ ]:
service.delete()